In [15]:
# !pip install torch==1.13.1+cpu torchvision==0.14.1+cpu -f https://download.pytorch.org/whl/cpu/torch_stable.html

# !pip install autogluon

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from autogluon.tabular import TabularDataset, TabularPredictor
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from scipy.signal import lfilter

In [17]:
df = pd.read_csv("task2_updated.csv")

In [18]:
df.shape

(33358, 176)

In [19]:
df.describe()

,mfcc_1,mfcc_2,mfcc_3,mfcc_4,mfcc_5,mfcc_6,mfcc_7,mfcc_8,mfcc_9,mfcc_10,...,tonnetz_3,tonnetz_4,tonnetz_5,tonnetz_6,hnr,jitter,shimmer,pitch_mean,energy_mean,label
count,33358.000000,33358.000000,33358.000000,33358.000000,33358.000000,33358.000000,33358.000000,33358.000000,33358.000000,33358.000000,...,33358.000000,33358.000000,33358.000000,33358.000000,33358.000000,33358.000000,33358.0,33358.000000,33358.000000,33358.000000
mean,-464.606001,81.195411,7.601951,22.660766,-4.784132,-7.919978,-10.231997,-12.413724,-8.225023,-3.659635,...,-0.005221,0.016709,0.005602,-0.009964,-92.321408,0.022731,0.0,1370.255993,0.067367,0.498591
std,35.924658,16.666366,9.186046,8.748914,9.175545,12.149602,8.589240,7.987213,8.238333,4.511582,...,0.045495,0.052887,0.015735,0.015453,30.990881,0.005889,0.0,202.580143,0.019495,0.500006
min,-652.506470,10.631331,-39.547359,-26.685741,-55.520386,-46.895409,-43.242626,-43.214661,-34.324177,-23.948406,...,-0.254249,-0.229005,-0.065296,-0.080615,-186.777218,0.006908,0.0,287.168549,0.017346,0.000000
25%,-483.513901,69.931112,1.807938,16.826989,-11.614328,-18.745957,-16.933169,-18.971106,-14.980084,-6.605721,...,-0.034043,-0.019985,-0.005113,-0.019798,-117.564670,0.019057,0.0,1256.397339,0.053571,0.000000
50%,-462.563293,81.233856,7.816589,22.455782,-6.001999,-6.651547,-10.794845,-12.112967,-7.946251,-3.413793,...,-0.003692,0.013476,0.004666,-0.009191,-88.436221,0.021558,0.0,1389.305420,0.062553,0.000000
75%,-442.570381,92.377342,13.631400,28.294442,2.021287,2.730283,-3.821682,-6.172355,-1.329778,-0.505511,...,0.024989,0.051946,0.015675,0.000302,-67.423593,0.024923,0.0,1504.960938,0.077495,1.000000
max,-243.866760,150.680740,52.349846,63.721615,36.332645,21.817154,18.182215,10.858657,14.706846,14.377119,...,0.222025,0.271541,0.084240,0.075496,4.559078,0.101724,0.0,2103.436523,0.178848,1.000000


In [20]:
df.isnull().sum()

mfcc_1         0
mfcc_2         0
mfcc_3         0
mfcc_4         0
mfcc_5         0
              ..
jitter         0
shimmer        0
pitch_mean     0
energy_mean    0
label          0
Length: 176, dtype: int64

In [21]:
df =df.dropna()

In [22]:
train_data = TabularDataset(df)
train_data.head()

,mfcc_1,mfcc_2,mfcc_3,mfcc_4,mfcc_5,mfcc_6,mfcc_7,mfcc_8,mfcc_9,mfcc_10,...,tonnetz_3,tonnetz_4,tonnetz_5,tonnetz_6,hnr,jitter,shimmer,pitch_mean,energy_mean,label
0,-448.422180,80.993530,13.313732,28.502640,-3.147002,-2.964426,-7.545156,-11.802809,2.450454,-3.691187,...,-0.100786,0.016575,-0.026689,-0.037001,-100.285502,0.018315,0.0,1229.895630,0.085697,1
1,-472.883942,95.293030,12.347318,14.079719,-21.829525,-24.046385,-7.827004,-12.601543,-20.181137,-8.110243,...,0.024803,0.050083,-0.013369,0.010755,-80.472091,0.017956,0.0,1285.848267,0.056584,0
2,-467.518555,92.519821,12.026536,9.413016,-5.435178,-15.849677,-18.588631,-17.920841,-14.868905,0.043479,...,0.011268,0.044513,0.012128,-0.012972,-63.139712,0.023886,0.0,1191.719971,0.056804,1
3,-452.814545,111.328484,11.075777,26.594025,15.664600,10.281798,11.363366,-5.489561,-6.835721,5.703945,...,-0.095303,0.044136,0.012801,-0.010281,-110.810515,0.016735,0.0,1229.298584,0.101746,1
4,-445.321228,92.499107,2.627282,11.540885,-9.992284,-15.990433,-18.050400,-19.711130,-18.808413,-1.209301,...,0.023026,-0.006950,0.000516,0.001171,-72.354565,0.021402,0.0,1192.956909,0.063783,1


In [23]:
predictor = TabularPredictor(label='label').fit(
    train_data,
    presets='best_quality',         
    excluded_model_types=['KNN']    
)

No path specified. Models will be saved in: "AutogluonModels/ag-20241002_112538"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #45~22.04.1-Ubuntu SMP PREEMPT_DYNAMIC Wed Sep 11 15:25:05 UTC 2
CPU Count:          72
Memory Avail:       236.59 GB / 251.57 GB (94.0%)
Disk Space Avail:   71.52 GB / 1373.98 GB (5.2%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of A

In [24]:
predictor.leaderboard(silent=True)

,model,score_val,eval_metric,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L3,0.911775,accuracy,71.349121,2360.974410,0.002180,0.788625,3,True,26
1,LightGBMXT_BAG_L2,0.911266,accuracy,58.629531,1704.484079,0.264788,15.992427,2,True,14
2,XGBoost_BAG_L2,0.910067,accuracy,59.292748,1791.464571,0.928005,102.972919,2,True,22
3,LightGBM_BAG_L2,0.908748,accuracy,58.638888,1705.942854,0.274145,17.451202,2,True,15
4,CatBoost_BAG_L2,0.908028,accuracy,58.562620,1725.897982,0.197877,37.406330,2,True,18
5,CatBoost_r177_BAG_L2,0.907968,accuracy,58.510866,1703.013074,0.146123,14.521422,2,True,25
6,LightGBMLarge_BAG_L2,0.906469,accuracy,58.777474,1783.912225,0.412730,95.420573,2,True,24
7,NeuralNetFastAI_BAG_L2,0.905840,accuracy,62.483922,1865.472259,4.119179,176.980607,2,True,21
8,NeuralNetTorch_BAG_L2,0.904970,accuracy,65.760824,2046.788630,7.396081,358.296978,2,True,23
9,RandomForestGini_BAG_L2,0.895917,accuracy,62.278750,1693.773108,3.914006,5.281456,2,True,16


In [25]:
predictor.fit_summary()

*** Summary of fit() ***
Estimated performance of each model:
                      model  score_val eval_metric  pred_time_val     fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0       WeightedEnsemble_L3   0.911775    accuracy      71.349121  2360.974410                0.002180           0.788625            3       True         26
1         LightGBMXT_BAG_L2   0.911266    accuracy      58.629531  1704.484079                0.264788          15.992427            2       True         14
2            XGBoost_BAG_L2   0.910067    accuracy      59.292748  1791.464571                0.928005         102.972919            2       True         22
3           LightGBM_BAG_L2   0.908748    accuracy      58.638888  1705.942854                0.274145          17.451202            2       True         15
4           CatBoost_BAG_L2   0.908028    accuracy      58.562620  1725.897982                0.197877          37.406330            2       True        

/home/aditya/notebooks/venv/lib/python3.10/site-packages/autogluon/core/utils/plots.py:169: UserWarning: AutoGluon summary plots cannot be created because bokeh is not installed. To see plots, please do: "pip install bokeh==2.0.1"
  warnings.warn('AutoGluon summary plots cannot be created because bokeh is not installed. To see plots, please do: "pip install bokeh==2.0.1"')


{'model_types': {'LightGBMXT_BAG_L1': 'StackerEnsembleModel_LGB',
  'LightGBM_BAG_L1': 'StackerEnsembleModel_LGB',
  'RandomForestGini_BAG_L1': 'StackerEnsembleModel_RF',
  'RandomForestEntr_BAG_L1': 'StackerEnsembleModel_RF',
  'CatBoost_BAG_L1': 'StackerEnsembleModel_CatBoost',
  'ExtraTreesGini_BAG_L1': 'StackerEnsembleModel_XT',
  'ExtraTreesEntr_BAG_L1': 'StackerEnsembleModel_XT',
  'NeuralNetFastAI_BAG_L1': 'StackerEnsembleModel_NNFastAiTabular',
  'XGBoost_BAG_L1': 'StackerEnsembleModel_XGBoost',
  'NeuralNetTorch_BAG_L1': 'StackerEnsembleModel_TabularNeuralNetTorch',
  'LightGBMLarge_BAG_L1': 'StackerEnsembleModel_LGB',
  'CatBoost_r177_BAG_L1': 'StackerEnsembleModel_CatBoost',
  'WeightedEnsemble_L2': 'WeightedEnsembleModel',
  'LightGBMXT_BAG_L2': 'StackerEnsembleModel_LGB',
  'LightGBM_BAG_L2': 'StackerEnsembleModel_LGB',
  'RandomForestGini_BAG_L2': 'StackerEnsembleModel_RF',
  'RandomForestEntr_BAG_L2': 'StackerEnsembleModel_RF',
  'CatBoost_BAG_L2': 'StackerEnsembleModel_

In [26]:
test_df = pd.read_csv("notebooks/task2_test.csv")

In [27]:
test_df.head()

,mfcc_1,mfcc_2,mfcc_3,mfcc_4,mfcc_5,mfcc_6,mfcc_7,mfcc_8,mfcc_9,mfcc_10,...,tonnetz_3,tonnetz_4,tonnetz_5,tonnetz_6,hnr,jitter,shimmer,pitch_mean,energy_mean,file_name
0,-486.401367,97.075951,1.455856,17.591446,-5.368714,-14.828186,-20.144354,-20.623831,-12.177615,1.159349,...,-0.024556,-0.015757,-0.000616,0.012106,-91.268641,0.021248,0.0,1386.938110,0.041484,0io4os2t_9.wav
1,-424.335236,65.030983,14.681046,16.013525,-10.309523,-15.890195,-22.942871,-26.800383,-15.417969,-5.794672,...,0.011504,-0.030832,-0.023195,-0.006103,-54.479123,0.018641,0.0,1497.434448,0.074718,jstty4wb_9.wav
2,-489.810303,82.875648,10.494175,34.404476,12.137540,2.209968,-3.565830,-9.362229,-1.637606,-2.134882,...,0.024558,0.045511,0.011326,-0.019273,-135.208743,0.030750,0.0,1303.233643,0.064343,exx4f0md_9.wav
3,-431.635590,74.653374,-8.876698,21.714447,-17.742594,-2.264153,-7.566887,-6.392578,-6.988452,-10.721841,...,-0.058158,0.085846,0.014873,0.013134,-86.537214,0.016136,0.0,1340.016968,0.095171,qeona02l_9.wav
4,-440.849945,88.376289,9.480526,10.677132,-19.611328,-15.447055,-11.921895,-21.429268,-18.442308,-9.003101,...,-0.022206,0.023161,-0.004883,-0.022752,-53.351956,0.017419,0.0,1387.257446,0.065176,n1ea0fwd_9.wav


In [28]:
test_df.drop(columns=['file_name'], inplace=True)
print(test_df)

           mfcc_1      mfcc_2     mfcc_3     mfcc_4     mfcc_5     mfcc_6  \
0     -486.401367   97.075951   1.455856  17.591446  -5.368714 -14.828186   
1     -424.335236   65.030983  14.681046  16.013525 -10.309523 -15.890195   
2     -489.810303   82.875648  10.494175  34.404476  12.137540   2.209968   
3     -431.635590   74.653374  -8.876698  21.714447 -17.742594  -2.264153   
4     -440.849945   88.376289   9.480526  10.677132 -19.611328 -15.447055   
...           ...         ...        ...        ...        ...        ...   
14291 -486.614532   76.802483  17.531792  17.700487 -15.644818 -17.676405   
14292 -396.636047  111.757584   0.855609  29.780388 -16.669813   6.712504   
14293 -417.486938   86.425224   6.044201  14.311959  -6.190600  -3.043458   
14294 -463.433258   67.831436   8.384674  24.028437  -5.456697   2.404174   
14295 -534.283386   84.228622  -5.355438  29.298330  11.741174   4.437322   

          mfcc_7     mfcc_8     mfcc_9    mfcc_10  ...  tonnetz_2  tonnetz_

In [29]:
predictions = predictor.predict(test_df)

print(predictions)

0        0
1        0
2        0
3        1
4        0
        ..
14291    0
14292    1
14293    1
14294    0
14295    0
Name: label, Length: 14296, dtype: int64


In [30]:
final_df = pd.DataFrame({
    'prediction': predictions
})

In [31]:
final_df.to_csv('task2_result.csv', index=False,header=False)

In [32]:
# predictor.feature_importance(data=train_data, model='WeightedEnsemble_L4')